### Solace Relativity v1.0 - Build Relativity and EJ Weighting

Read exploits db, use derivedValueIndicators and build data-values files and relativity scores, build expert-judgement template files, and build expert-judgement weighting files.

In [17]:
import pandas as pd
import json
import numpy as np
import math
import os

#WIP

## Load Exploits, Protocols(tbd), and mappingTables (transactional and temporal) 
temporalDb=pd.read_excel('../temporalDB/TemporalDataset.xlsx')
exploitsDf = pd.read_csv('../exploits/historical_exploits.csv')
f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()

# Init dfs for lookup tables
tvlSolaceLookupTable = pd.DataFrame(lookupTables['tvlSolaceLookup'])
auditsSolaceLookupTable = pd.DataFrame(lookupTables['auditSolaceLookup'])
oraclesSolaceLookupTable = lookupTables['oracleSolaceLookup'] # just values, not mappings: yes/no/unknown
categorySolaceLookupTable=pd.DataFrame(lookupTables['categorySolaceLookup'])



# Calc Exploit Damage ratio
exploitsDf['tvlPreExploit'] = exploitsDf['tvlPreExploit'] / 10 
exploitsDf['exploitDamage'] = exploitsDf['loss'] / exploitsDf['tvlPreExploit']


# Calc time to exploit
exploitsDf[['launchDate','exploitDate']] = exploitsDf[['launchDate','exploitDate']].apply(pd.to_datetime)
exploitsDf['timeToExploit'] = (exploitsDf['exploitDate'] - exploitsDf['launchDate']).dt.days
exploitsDf['timeToExploit'] = exploitsDf['timeToExploit'].astype(object).replace(np.nan, 'null')


# Derive tvlSolace column
def lookup_tvlSolace(tvl):
    if math.isnan(tvl):
        return 'null'
    match = (tvlSolaceLookupTable['lowerLimit'] <= tvl) & (tvlSolaceLookupTable['upperLimit'] > tvl)
    tvlOut = tvlSolaceLookupTable['tvlSolace'][match]
    return tvlOut.values[0]


# Derive auditSolace column
def lookup_auditSolace(audits):
    if math.isnan(audits):
        return 'Unknown'
    match = (auditsSolaceLookupTable['auditsOg'] == audits)
    auditsOut = auditsSolaceLookupTable['auditSolace'][match]
    return auditsOut.values[0]


# Derive oracleSolace column
def lookup_oracleSolace(oracles,catalog):
    if pd.isna(catalog):
        return 'Unknown'
    elif pd.isna(oracles):
        return 'No'
    else:
        return "Yes"


#Solace judgement

exploitsDf['tvlSolace'] = exploitsDf['tvlPreExploit'].apply(lookup_tvlSolace)
temporalDb['tvlSolace'] = temporalDb['tvl'].apply(lookup_tvlSolace)
exploitsDf['auditSolace'] = exploitsDf['audits'].apply(lookup_auditSolace)
temporalDb['auditSolace'] = temporalDb['audits'].apply(lookup_auditSolace)            
exploitsDf['oracleSolace'] = exploitsDf.apply(lambda x: lookup_oracleSolace(x.oracles, x.category), axis=1)
temporalDb['oracleSolace'] = temporalDb.apply(lambda x: lookup_oracleSolace(x.oracles, x.tags), axis=1)


# Create columns for use in one way calcs
exploitsDf['tvlAvgDamage'] = exploitsDf.groupby(['tvlSolace'])['exploitDamage'].transform('mean')
exploitsDf['auditAvgDamage'] = exploitsDf.groupby(['auditSolace'])['exploitDamage'].transform('mean')
exploitsDf['oracleAvgDamage'] = exploitsDf.groupby(['oracleSolace'])['exploitDamage'].transform('mean')

averageDamage = exploitsDf['exploitDamage'].mean()

exploitsDf['tvlDamageRatio'] = exploitsDf['tvlAvgDamage'] / averageDamage
exploitsDf['auditDamageRatio'] = exploitsDf['auditAvgDamage'] / averageDamage
exploitsDf['oracleDamageRatio'] = exploitsDf['oracleAvgDamage'] / averageDamage

print('Average exploit damage ratio: ',averageDamage)

# Create one way 'tables', separate for readability 
# TVL
tvlOneWay = pd.DataFrame()
tvlOneWay['damageRatio'] = exploitsDf['tvlDamageRatio'].unique()
tvlOneWay['tvlSolace'] =exploitsDf['tvlSolace'].unique()
tvlOneWay['avgDamage'] = exploitsDf['tvlAvgDamage'].unique()

# AUDITS
auditOneWay = pd.DataFrame()
auditOneWay['damageRatio'] = exploitsDf['auditDamageRatio'].unique()
auditOneWay['auditSolace'] = exploitsDf['auditSolace'].unique()
auditOneWay['avgDamage'] = exploitsDf['auditAvgDamage'].unique()

# ORACLES
oracleOneWay = pd.DataFrame()
oracleOneWay['damageRatio'] = exploitsDf['oracleDamageRatio'].unique()
oracleOneWay['oracleSolace'] =exploitsDf['oracleSolace'].unique()
oracleOneWay['avgDamage'] = exploitsDf['oracleAvgDamage'].unique()

def write_json(df, fileName, drop):
    if drop == True:
        df.drop(['avgDamage'], axis=1, inplace=True)
    data = df.to_json(orient='columns')
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    with open(fileName, 'w') as outfile:
        outfile.write(data)

write_json(pd.merge(tvlOneWay, tvlSolaceLookupTable, how="right", on='tvlSolace'), '../governance/data-values/tvlDataValues.json', True)
write_json(pd.merge(auditOneWay, auditsSolaceLookupTable, how="right", on='auditSolace'), '../governance/data-values/auditDataValues.json', True)
write_json(oracleOneWay, '../governance/data-values/oracleDataValues.json', True)



print("Data values json files created in governance directory")

# Initialize files and weightings

# EJ weighting files
def create_weights(predictor):
    weightingsTemplate = '{"' + predictor + 'Basis": {"1": "data","2":"Ej1"},"weight": {"1": 0, "2":1}}'
    weightingsTemplateJson = json.loads(weightingsTemplate)
    weightingsTemplateDf = pd.DataFrame(weightingsTemplateJson)
    write_json(weightingsTemplateDf, '../governance/weightings/'+predictor+'Weighting.json', False)
    
create_weights('audit')
create_weights("oracle")
create_weights("tvl")
create_weights("category")




Average exploit damage ratio:  0.24146191913870418
Data values json files created in governance directory


C:\Users\haral\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


### Temporal Data relativities

In [18]:
# Loading lookup tables
def open_data(file):
    Data = open(file)
    lookupTables = json.loads(Data.read())
    Data.close()
    return lookupTables
# Append relativity columns to the temporalDB
def add_column(file,database,merger,newKey):
    lookup=open_data(file)
    attributeSolaceRelativity=lookup
    attributeFrame=pd.DataFrame(attributeSolaceRelativity)
    attributeFrame=attributeFrame.rename(columns={'damageRatio':newKey})
    combinedAuditTable1=pd.merge(database,attributeFrame,left_on=merger, right_on=merger, how='left')
    return combinedAuditTable1
# Calculate final relativity based on the weighting for each Ej and data values
def calc_finalRelativity(weightLookup,combinedtable2,column1,column2,key,key_weight):
    lookupTable=open_data(weightLookup)
    lookupFrame=pd.DataFrame(lookupTable)
    data_weight=lookupFrame[lookupFrame[key_weight]=='data']['weight']
    solaceRM_weight=lookupFrame[lookupFrame[key_weight]=='Ej1']['weight']
    combinedtable2[key]=(column1*data_weight.values)+(column2*solaceRM_weight.values) #+(column1*solaceRM_weight.values)+(column2*solaceRM_weight.values)
    return combinedtable2
def aggregateTables(weightLookup,file,file2,finalKey,key1,key2,database,merger,key_weight):
    column_test=add_column(file,database,merger,key1)
    column_test2=add_column(file2,column_test,merger,key2)
    column_test3=calc_finalRelativity(weightLookup,column_test2,column_test2[key1],column_test2[key2],finalKey,key_weight)
    return column_test3

auditSolaceRelativities=aggregateTables('../governance/weightings/auditWeighting.json','../governance/data-values/auditRelativities.json','../governance/expert-judgement/solaceRM/ejAuditDataValues.json','auditFinalRelativity','relativityAuditData','relativityAuditEj1',temporalDb,'auditSolace','auditBasis')
tvlSolaceRelativities=aggregateTables('../governance/weightings/tvlWeighting.json','../governance/data-values/tvlRelativities.json','../governance/expert-judgement/solaceRM/ejTvlDataValues.json','tvlFinalRelativity','relativityTvlData','relativityTvlEj1',auditSolaceRelativities,'tvlSolace','tvlBasis')
oracleSolaceRelativities=aggregateTables('../governance/weightings/oracleWeighting.json','../governance/data-values/oracleRelativities.json','../governance/expert-judgement/solaceRM/ejOracleDataValues.json','oracleFinalRelativity','relativityOracleData','relativityOracleEj1',tvlSolaceRelativities,'oracleSolace','oracleBasis')

oracleSolaceRelativities['finalRelativity']=oracleSolaceRelativities['auditFinalRelativity']*oracleSolaceRelativities['tvlFinalRelativity']*oracleSolaceRelativities['oracleFinalRelativity']

oracleSolaceRelativities

,appId,tags,name,categorySolace,SolaceID,address,symbol,audits,chains,oracles,...,lowerLimit_x,upperLimit_x,relativityTvlEj1,lowerLimit_y,upperLimit_y,tvlFinalRelativity,relativityOracleData,relativityOracleEj1,oracleFinalRelativity,finalRelativity
0,aave-amm,lending,Aave AMM,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],...,10000000001,100000000000,1,10000000001,100000000000,1.0,0.671936,1,1.0,1.0
1,aave-safety-module,liquidity-pool,Aave Safety Module,liquidity-pool,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],...,10000000001,100000000000,1,10000000001,100000000000,1.0,0.671936,1,1.0,1.0
2,aave,lending,Aave,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],...,10000000001,100000000000,1,10000000001,100000000000,1.0,0.671936,1,1.0,1.0
3,aave-v2,lending,Aave V2,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],...,10000000001,100000000000,1,10000000001,100000000000,1.0,0.671936,1,1.0,1.0
4,aavegotchi,yield-aggregator,Aavegotchi,yield-aggregator,aavegotchi,0x3f382dbd960e3a9bbceae22651e88158d2791550,GHST,2.0,"['Polygon', 'Ethereum']",NaN,...,0,100000000,3,0,100000000,3.0,0.934027,1,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,xtoken,asset-management,xToken,asset-management,xtoken,0x7f3edcdd180dbe4819bd98fee8929b5cedb3adeb,XTK,2.0,['Ethereum'],NaN,...,0,100000000,3,0,100000000,3.0,0.934027,1,1.0,3.0
124,yam,asset-management,Yam,asset-management,yam,0x0aacfbec6a24756c20d41914f2caba817c0d8521,YAM,3.0,['Ethereum'],NaN,...,0,100000000,3,0,100000000,3.0,0.934027,1,1.0,3.0
125,yaxis,yield-aggregator,yAxis,yield-aggregator,yaxis,0x0ada190c81b814548ddc2f6adc4a689ce7c1fe73,YAXIS,2.0,['Ethereum'],NaN,...,0,100000000,3,0,100000000,3.0,0.934027,1,1.0,3.0
126,yearn,yield-aggregator,Yearn,yield-aggregator,yearn,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,YFI,1.0,"['Ethereum', 'Fantom']",NaN,...,1000000001,10000000000,1,1000000001,10000000000,1.0,0.934027,1,1.0,1.0


### Map protocols to tiers based on final Relativity

In [19]:
RelativityTable=pd.read_json('../governance/reference/relativityTable.json')

#Map into tiers based on relativity
def tiers(relativities,RelativityTable):
    min_val=RelativityTable['minRelativity']
    max_val=RelativityTable['maxRelativity']
    tier=RelativityTable['tier']
    riskTiers=[]
    for value in relativities:
        if value<=max_val[0] and value>=min_val[0]:
                riskTiers.append(tier[0])
        elif value<=max_val[1] and value>=min_val[1]:
                riskTiers.append(tier[1])
        elif value<=max_val[2] and value>=min_val[2]:
                riskTiers.append(tier[2])
        else:
                riskTiers.append(tier[3])
    return riskTiers



protocolRelativity=oracleSolaceRelativities['finalRelativity']
Risk_Tiers=tiers(protocolRelativity,RelativityTable)
oracleSolaceRelativities['tier']=Risk_Tiers

protocolMap=oracleSolaceRelativities.filter(['appId','tier','categorySolace'], axis=1)
protocolMap=protocolMap.rename(columns={'categorySolace':'category'})

#Create the Protocol Map and append it to the Solace RM workbook
with pd.ExcelWriter('../governance/reference/SolaceRMValues.xlsx', mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:protocolMap.to_excel(writer, sheet_name="protocolMap")

